<a href="https://colab.research.google.com/github/gigajet/transformer/blob/master/running_fairseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparations

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# Google Colab doesn't support `ln`.

Mounted at /content/drive


In [2]:
# Install fairseq
!git clone https://github.com/pytorch/fairseq
%cd fairseq
!pip install --editable ./

Cloning into 'fairseq'...
remote: Enumerating objects: 31273, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 31273 (delta 18), reused 45 (delta 18), pack-reused 31225
Receiving objects: 100% (31273/31273), 21.53 MiB | 15.61 MiB/s, done.
Resolving deltas: 100% (23055/23055), done.
/content/fairseq
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 123 kB 12.0 MB/s 
     |████████████████████████████████| 90 kB 8.6 MB/s 
     |████████████████████████████████| 236 kB 48.6 MB/s 
     |████████████████████████████████| 112 kB 57.7 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=e76f532b39ec7272f3a65ceee14

In [3]:
# Dependencies for preprocessing
!pip install fastBPE sacremoses subword_nmt

     |████████████████████████████████| 880 kB 18.0 MB/s 
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483164 sha256=3d21ad62e7dbba248531515a9287ab9211022fd196aa700aee5e680eb56b808c
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e685ad23ec88de4592b31633a5727df9b9374c0555dd25ba24de1e2fe1d18119
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built fastBPE sacremoses


In [4]:
# Run this if you get error "importlib_metadata.PackageNotFoundError: No package metadata was found for fairseq" in next cell
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

## Verify that fairseq is now usable (optional)

In [ ]:
# This cell is for verifying the install
# Interactive translation
import torch
import fairseq

# List available models
torch.hub.list('pytorch/fairseq')  # [..., 'transformer.wmt16.en-de', ... ]

# Load a transformer trained on WMT'16 En-De
# Note: WMT'19 models use fastBPE instead of subword_nmt, see instructions below
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt16.en-de',
                       tokenizer='moses', bpe='subword_nmt')
en2de.eval()  # disable dropout

# The underlying model is available under the *models* attribute
assert isinstance(en2de.models[0], fairseq.models.transformer.TransformerModel)

# Move model to GPU for faster translation
# en2de.cuda()

# Translate a sentence
en2de.translate('Hello world!')
# 'Hallo Welt!'

# Batched translation
en2de.translate(['Hello world!', 'The cat sat on the mat.'])
# ['Hallo Welt!', 'Die Katze saß auf der Matte.']

2022-05-02 04:29:06 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Downloading: "https://github.com/pytorch/fairseq/archive/main.zip" to /root/.cache/torch/hub/main.zip
Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
2022-05-02 04:29:10 | INFO | fairseq.file_utils | https://dl.fbaipublicfiles.com/fairseq/models/wmt16.en-de.joined-dict.transformer.tar.bz2 not found in cache, downloading to /tmp/tmpy9uuosu6
  7%|▋         | 150277120/2193287384 [00:05<01:50, 18416384.24B/s]

KeyboardInterrupt: ignored

## Prepare & Train IWSLT'14 German to English (Transformer)

In [ ]:
# Download and prepare the data
%cd examples/translation/
!bash prepare-iwslt14.sh
%cd ../..

/content/fairseq/examples/translation
Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
^C
/content/fairseq


In [ ]:
# Preprocess/binarize the data
# Look below on how to set environment variable in GG Colab
%env TEXT=examples/translation/iwslt14.tokenized.de-en
!fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir /content/drive/MyDrive/translation/iwslt14.tokenized.de-en \
    --workers 20


env: TEXT=examples/translation/iwslt14.tokenized.de-en
/bin/bash: fairseq-preprocess: command not found


In [ ]:
# Train the model, please change --max-epoch depends whether you use gpu/cpu
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt14.tokenized.de-en \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 15 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir /content/drive/MyDrive/translation/iwslt14.tokenized.de-en/checkpoints


env: CUDA_VISIBLE_DEVICES=0
/bin/bash: fairseq-train: command not found


In [ ]:
# Evaluate our model
!fairseq-generate /content/drive/MyDrive/translation/iwslt14.tokenized.de-en \
    --path /content/drive/MyDrive/translation/iwslt14.tokenized.de-en/checkpoints/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe

/bin/bash: fairseq-generate: command not found


## Prepare IWSLT'15 English - Vietnamese data
Data is already in Drive `/content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset`, unnecessary

In [36]:
!unzip /content/drive/MyDrive/translation/en-vi.zip -d /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset
# !cp /content/drive/MyDrive/translation/myprepare-iwslt15-en-vi.sh .

Archive:  /content/drive/MyDrive/translation/en-vi.zip
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.dev2010.en-vi.en  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.dev2010.en-vi.vi  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2010.en-vi.en  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2010.en-vi.vi  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2011.en-vi.en  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2011.en-vi.vi  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2012.en-vi.en  
  inflating: /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset/IWSLT15.TED.tst2012.en-vi.vi  
  inflating: /content/drive/MyDrive/translation/iwslt15.t

In [37]:
#!sh myprepare-iwslt15-en-vi.sh
# After this, we have `train valid test` in iwslt15.tokenized.en-vi

In [38]:
# Preprocess/binarize the data
# Look below on how to set environment variable in GG Colab
# %env TEXT=iwslt15.tokenized.en-vi
# !fairseq-preprocess --source-lang en --target-lang vi \
#     --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
#     --destdir /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
#     --workers 20

## Train IWSLT'15 English-Vietnamese (Transformer)

In [ ]:
# arch transformer, ở đây dùng transformer_iwslt_de_en là dùng lại architecture của nó
# Train the model, please change --max-epoch depends whether you use gpu/cpu
# see stop-time-hours
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 36 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir checkpoints
    # --save-dir /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/checkpoints


env: CUDA_VISIBLE_DEVICES=0
/bin/bash: fairseq-train: command not found


In [41]:
# Evaluate our model, change the path
!fairseq-generate /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
    --path /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/checkpoints-nntransformer/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe \
    --user-dir /content/mymodel

2022-05-07 15:48:42 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-07 15:48:45 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/content/mymodel', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 're

## Try to add FuzzyLayer to Transformer


In [ ]:
# FuzzyLayer
import torch
import math
from torch import nn

class FuzzyLayer(nn.Module):

    def __init__(self, output_dim: int, **kwargs):
        super(FuzzyLayer, self).__init__(**kwargs)
        self.output_dim = output_dim
        self.fuzzy_degree = nn.Parameter(torch.empty(output_dim))
        self.sigma = nn.Parameter(torch.ones(output_dim))
        self.reset_parameters()
        
    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(in_features), 1/sqrt(in_features)). For details, see
        # https://github.com/pytorch/pytorch/issues/57109
        nn.init.uniform_(self.fuzzy_degree)

    def forward(self, input, **kwargs):
        x = torch.repeat_interleave(torch.unsqueeze(input,-1), self.output_dim, dim=-1)
        fuzzy_out = torch.exp(
                      -torch.sum(
                          torch.square((x-self.fuzzy_degree)/(self.sigma**2))            
                          ,dim=-2, keepdims=False)
              )
        return fuzzy_out


class FuzzyRuleLayer(nn.Module):

    def __init__(self, input_dim: int, output_dim,**kwargs):
        super(FuzzyRuleLayer, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = nn.ModuleList([
            FuzzyLayer(output_dim) for _ in range(input_dim)
        ])

    def forward(self, input):
        batch_size, input_dim = input.size()
        an=torch.ones(batch_size, self.output_dim)
        for layer in self.layers:
            an=an*layer(input)
        return an

z=FuzzyRuleLayer(5,35)
y=torch.ones(3,5)
print(z(y).shape)
print(z(y))

torch.Size([3, 35])
tensor([[6.7112e-06, 6.8293e-04, 1.1360e-05, 1.4338e-05, 3.1253e-06, 4.4289e-04,
         1.2499e-04, 2.6953e-04, 5.7319e-06, 3.1216e-05, 5.5655e-05, 3.2802e-04,
         3.7528e-02, 1.1846e-05, 7.1205e-07, 2.0275e-06, 7.1357e-07, 4.7466e-04,
         1.9231e-06, 2.3387e-03, 8.4445e-06, 1.2212e-04, 1.2321e-04, 4.9719e-04,
         9.7817e-03, 7.2931e-03, 2.3511e-06, 5.0083e-02, 1.0146e-05, 1.0202e-06,
         5.4978e-04, 8.7630e-04, 9.0879e-02, 2.8873e-08, 5.0999e-03],
        [6.7112e-06, 6.8293e-04, 1.1360e-05, 1.4338e-05, 3.1253e-06, 4.4289e-04,
         1.2499e-04, 2.6953e-04, 5.7319e-06, 3.1216e-05, 5.5655e-05, 3.2802e-04,
         3.7528e-02, 1.1846e-05, 7.1205e-07, 2.0275e-06, 7.1357e-07, 4.7466e-04,
         1.9231e-06, 2.3387e-03, 8.4445e-06, 1.2212e-04, 1.2321e-04, 4.9719e-04,
         9.7817e-03, 7.2931e-03, 2.3511e-06, 5.0083e-02, 1.0146e-05, 1.0202e-06,
         5.4978e-04, 8.7630e-04, 9.0879e-02, 2.8873e-08, 5.0999e-03],
        [6.7112e-06, 6.8293e-0

## Try to add MyLSTM to fairseq (CPU only)


### Some notes
+ Follow the tutorial at https://fairseq.readthedocs.io/en/latest/tutorial_simple_lstm.html, add `@register_model` and `@register_model_architecture` to appropriate class.

+ That's not all, you must either:

1. Move your `model_name.py` to `fairseq/models`

2. Inside your directory (supposed `user_dir`), create a new folder named `models` and move your file there. In the command line (such as `fairseq-train`), specify `--user-dir /path/to/user_dir`

Because each time the command line tool runs, it must run some bootstrap code to get the current model list, parse your arguments, etc. Therefore you must follow some of their rules.

In [ ]:
# Make mymodel path and prepare my_model (in this case `MyLSTM.py`) in user_dir (in this case `mymodel`)
%cd /content
!rm -rf my_transformer
!git clone https://github.com/gigajet/transformer my_transformer
!mkdir mymodel
%cd mymodel
!mkdir models
%cd models 
!cp /content/my_transformer/MyFairseqLSTM.py .
!cp -r /content/my_transformer/layer .


/content
fatal: destination path 'my_transformer' already exists and is not an empty directory.
mkdir: cannot create directory ‘mymodel’: File exists
/content/mymodel
mkdir: cannot create directory ‘models’: File exists
/content/mymodel/models


In [ ]:
# Train the model, please change --max-epoch depends whether you use gpu/cpu
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt14.tokenized.de-en \
    --encoder-dropout 0.2 --decoder-dropout 0.2 \
    --optimizer adam --lr 0.005 --lr-shrink 0.5 \
    --max-epoch 1 \
    --max-tokens 12000 \
    --save-dir checkpoints \
    --user-dir /content/mymodel \
    --arch mylstm_default \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

env: CUDA_VISIBLE_DEVICES=0
2022-04-22 10:02:58 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-04-22 10:02:59 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-04-22 10:03:01 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/content/mymodel', 'empty_cache_freq': 0, 'all_gath

## Get mymodel from github and prepare directory structure that satisfies fairseq requirement. Adding custom model to fairseq in `/content/mymodel`

In [21]:
# Make mymodel path and prepare my_model (in this case `MyLSTM.py`) in user_dir (in this case `mymodel`)
%cd /content
!rm -rf my_transformer
!git clone https://github.com/gigajet/transformer my_transformer
!rm -rf mymodel
!mkdir mymodel
%cd mymodel
!cp -r /content/my_transformer models

# Xóa thể loại không liên quan
%cd models
!rm -f string_reverser_*.py

# Finalize
%cd /content/fairseq


/content
Cloning into 'my_transformer'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 267 (delta 13), reused 14 (delta 6), pack-reused 243
Receiving objects: 100% (267/267), 28.30 MiB | 7.17 MiB/s, done.
Resolving deltas: 100% (148/148), done.
/content/mymodel
/content/mymodel/models
/content/fairseq


In [22]:
# fix the import layer to mymodel.layer
# F*** PYTHON3 IMPORT
!find /content/mymodel -type f -exec sed -i "s/from layer\./from mymodel.models.layer./g" {} \;

##Training nntransformer_default

In [ ]:
# Use MyTransformer to train en-vi data
# Train the model, please change --max-epoch depends whether you use gpu/cpu
# see stop-time-hours
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
    --arch nntransformer_default \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 100 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir /content/drive/MyDrive/iwslt15.tokenized.en-vi/checkpoints-nntransformer \
    --user-dir /content/mymodel


env: CUDA_VISIBLE_DEVICES=0
2022-05-06 15:16:16 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-06 15:16:18 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/content/mymodel', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': No

##Training custom proposal transformer
Arch is `proposalX_default`

In [23]:
# Train the model, please change --max-epoch depends whether you use gpu/cpu
# see stop-time-hours
%env CUDA_VISIBLE_DEVICES=0 
!fairseq-train \
    /content/drive/MyDrive/translation/iwslt15.tokenized.en-vi/dataset \
    --arch proposal3_default \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-epoch 100 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir /content/drive/MyDrive/iwslt15.tokenized.en-vi/checkpoints-proposal-3 \
    --user-dir /content/mymodel


env: CUDA_VISIBLE_DEVICES=0
2022-05-07 15:07:26 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-05-07 15:07:29 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': '/content/mymodel', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': No